In [1]:
#Importing libraries
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp 
import sklearn
import scipy.stats as st
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from matplotlib import pyplot

#SVC met BA threshold for all sources so below are the RFE results for them

### AFFF-GW

In [2]:
data_rf = pd.read_csv(r'240617-NTA-AVG-EUC-30-CLOSEST-FEATURE-logT.csv', header=0) #Targets: 92 samples X 581 features
#del data_rf[data_rf.columns[0]] #Dropping sample information
#print(data_rf)

In [3]:
#Prompt user for source type of interest (AFFF-GW, LF, BSL, WWTP, PP or PG)
preferred_type = input("Enter the source type of interest: ")

Enter the source type of interest: AFFF-GW


In [4]:
#Manipulating data frame based on user input to make "Type" column read 1 for all samples of source of interest and 0 for all other samples
#Set up for binary classification (one-vs-all format)

# Define a function to apply to each row
def set_type(row):
    if row['Type'] == preferred_type:
        return 1
    else:
        return 0

# Create a new column "Type 2" with the updated values
data_rf['Type_2'] = data_rf.apply(set_type, axis=1)
del data_rf[data_rf.columns[0]] #Dropping original type column
#Reordering columns with Type_2 as first column
cols = list(data_rf.columns)
cols = [cols[-1]] + cols[:-1]
data_rf = data_rf[cols]

# Save the updated DataFrame to a new CSV file (if needed)
data_rf.to_csv('sample_data_with_labels_NEW10.csv', index=False)

In [5]:
#Changing pandas data frame to numpy for use in ML
data_rf_np = data_rf.to_numpy()
target_1 = data_rf_np[:,0].reshape(-1,1) #Convert target variables to 2D-array for sci-kit learn
data_1 = data_rf_np[:,1:]

#class_names=np.array([0.0,1.0])
#print(data_1.shape)
print(data_1)
#data_1 = pd.DataFrame(data_1)
#data_1.to_csv('log10_dat.csv', index=False)

[[2.54570928 2.54570928 4.11539313 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 4.54451372 4.87498812 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 3.22038648 4.8459347  ... 2.54570928 2.54570928 2.54570928]
 ...
 [2.54570928 2.54570928 2.54570928 ... 3.02475995 2.54570928 2.54570928]
 [2.54570928 2.54570928 2.54570928 ... 3.2965849  2.54570928 2.93484187]
 [2.54570928 2.54570928 3.58667926 ... 3.33173239 2.54570928 2.54570928]]


In [6]:
#GW with SVC
from sklearn.svm import SVC
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(1, 31):
        rfe = RFE(estimator=SVC(kernel='linear',
                    C=16.40532, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=i)
        model = SVC(kernel='linear',
                    C=16.40532, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

#Evaluate model
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, data_1, target_1, scoring='balanced_accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [7]:
# Get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, data_1, target_1)
    results.append(scores)
    names.append(name)
    print('>%s Features: %d, Balanced Accuracy: %.3f (%.3f)' % (name, int(name), np.mean(scores), np.std(scores)))

>1 Features: 1, Balanced Accuracy: 0.594 (0.188)
>2 Features: 2, Balanced Accuracy: 0.685 (0.222)
>3 Features: 3, Balanced Accuracy: 0.710 (0.217)
>4 Features: 4, Balanced Accuracy: 0.722 (0.221)
>5 Features: 5, Balanced Accuracy: 0.782 (0.221)
>6 Features: 6, Balanced Accuracy: 0.799 (0.227)
>7 Features: 7, Balanced Accuracy: 0.821 (0.221)
>8 Features: 8, Balanced Accuracy: 0.808 (0.220)
>9 Features: 9, Balanced Accuracy: 0.810 (0.210)
>10 Features: 10, Balanced Accuracy: 0.839 (0.202)
>11 Features: 11, Balanced Accuracy: 0.858 (0.190)
>12 Features: 12, Balanced Accuracy: 0.860 (0.192)
>13 Features: 13, Balanced Accuracy: 0.863 (0.190)
>14 Features: 14, Balanced Accuracy: 0.863 (0.190)
>15 Features: 15, Balanced Accuracy: 0.861 (0.177)
>16 Features: 16, Balanced Accuracy: 0.859 (0.164)
>17 Features: 17, Balanced Accuracy: 0.865 (0.166)
>18 Features: 18, Balanced Accuracy: 0.844 (0.177)
>19 Features: 19, Balanced Accuracy: 0.842 (0.175)
>20 Features: 20, Balanced Accuracy: 0.844 (0.176

In [13]:
rfe = RFE(estimator=SVC(kernel='linear',
                    C=16.40532, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=5)
rfe.fit(data_1,target_1.ravel())
for i in range(data_1.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 19.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected False, Rank: 16.000
Column: 4, Selected False, Rank: 12.000
Column: 5, Selected False, Rank: 8.000
Column: 6, Selected False, Rank: 9.000
Column: 7, Selected False, Rank: 13.000
Column: 8, Selected False, Rank: 18.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 6.000
Column: 11, Selected False, Rank: 15.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 14.000
Column: 14, Selected False, Rank: 23.000
Column: 15, Selected False, Rank: 3.000
Column: 16, Selected False, Rank: 2.000
Column: 17, Selected False, Rank: 22.000
Column: 18, Selected False, Rank: 7.000
Column: 19, Selected False, Rank: 24.000
Column: 20, Selected False, Rank: 25.000
Column: 21, Selected False, Rank: 17.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 26.000
Column: 24, Selected False, Rank: 5.000
Co

In [14]:
# Load feature names from the second dataset
labels_dat = pd.read_csv("240128-NTA-Normalized-Labels-Feature.csv")
feature_names = labels_dat.columns

# Print selected features with their names
selected_feature_indices = [i for i in range(len(rfe.support_)) if rfe.support_[i]]

for index in selected_feature_indices:
    print('Feature Name: %s, Index: %d, Rank: %.3f' % (feature_names[index], index, rfe.ranking_[index]))

Feature Name: 390.964442/9.69, Index: 1, Rank: 1.000
Feature Name: 218.986076/7.21, Index: 2, Rank: 1.000
Feature Name: 341.160715/9.2, Index: 9, Rank: 1.000
Feature Name: 241.180448/9.53, Index: 12, Rank: 1.000
Feature Name: 132.056597/7.15, Index: 22, Rank: 1.000


In [ ]:
selected_features_df = pd.DataFrame({
    'Feature Name': [feature_names[index] for index in selected_feature_indices]
})

# Export the DataFrame to an Excel file
selected_features_df.to_excel("RFE_selected_features_norm_GW_SVC.xlsx", index=False)

### LL

In [15]:
data_rf = pd.read_csv(r'240617-NTA-AVG-EUC-30-CLOSEST-FEATURE-logT.csv', header=0) #Targets: 92 samples X 581 features
#del data_rf[data_rf.columns[0]] #Dropping sample information
#print(data_rf)

In [16]:
#Prompt user for source type of interest (AFFF-GW, LF, BSL, WWTP, PP or PG)
preferred_type = input("Enter the source type of interest: ")

Enter the source type of interest: LL


In [17]:
#Manipulating data frame based on user input to make "Type" column read 1 for all samples of source of interest and 0 for all other samples
#Set up for binary classification (one-vs-all format)

# Define a function to apply to each row
def set_type(row):
    if row['Type'] == preferred_type:
        return 1
    else:
        return 0

# Create a new column "Type 2" with the updated values
data_rf['Type_2'] = data_rf.apply(set_type, axis=1)
del data_rf[data_rf.columns[0]] #Dropping original type column
#Reordering columns with Type_2 as first column
cols = list(data_rf.columns)
cols = [cols[-1]] + cols[:-1]
data_rf = data_rf[cols]

# Save the updated DataFrame to a new CSV file (if needed)
data_rf.to_csv('sample_data_with_labels_NEW10.csv', index=False)

In [18]:
#Changing pandas data frame to numpy for use in ML
data_rf_np = data_rf.to_numpy()
target_1 = data_rf_np[:,0].reshape(-1,1) #Convert target variables to 2D-array for sci-kit learn
data_1 = data_rf_np[:,1:]

#class_names=np.array([0.0,1.0])
#print(data_1.shape)
print(data_1)
#data_1 = pd.DataFrame(data_1)
#data_1.to_csv('log10_dat.csv', index=False)

[[2.54570928 2.54570928 4.11539313 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 4.54451372 4.87498812 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 3.22038648 4.8459347  ... 2.54570928 2.54570928 2.54570928]
 ...
 [2.54570928 2.54570928 2.54570928 ... 3.02475995 2.54570928 2.54570928]
 [2.54570928 2.54570928 2.54570928 ... 3.2965849  2.54570928 2.93484187]
 [2.54570928 2.54570928 3.58667926 ... 3.33173239 2.54570928 2.54570928]]


In [19]:
#ll with SVC
from sklearn.svm import SVC
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(1, 31):
        rfe = RFE(estimator=SVC(kernel='linear',
                    C=1.0734, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=i)
        model = SVC(kernel='linear',
                    C=1.0734, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

#Evaluate model
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, data_1, target_1, scoring='balanced_accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [20]:
# Get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, data_1, target_1)
    results.append(scores)
    names.append(name)
    print('>%s Features: %d, Balanced Accuracy: %.3f (%.3f)' % (name, int(name), np.mean(scores), np.std(scores)))

>1 Features: 1, Balanced Accuracy: 0.691 (0.181)
>2 Features: 2, Balanced Accuracy: 0.746 (0.197)
>3 Features: 3, Balanced Accuracy: 0.801 (0.194)
>4 Features: 4, Balanced Accuracy: 0.860 (0.156)
>5 Features: 5, Balanced Accuracy: 0.873 (0.144)
>6 Features: 6, Balanced Accuracy: 0.887 (0.131)
>7 Features: 7, Balanced Accuracy: 0.887 (0.138)
>8 Features: 8, Balanced Accuracy: 0.865 (0.147)
>9 Features: 9, Balanced Accuracy: 0.885 (0.138)
>10 Features: 10, Balanced Accuracy: 0.914 (0.113)
>11 Features: 11, Balanced Accuracy: 0.917 (0.108)
>12 Features: 12, Balanced Accuracy: 0.915 (0.114)
>13 Features: 13, Balanced Accuracy: 0.924 (0.116)
>14 Features: 14, Balanced Accuracy: 0.928 (0.112)
>15 Features: 15, Balanced Accuracy: 0.926 (0.114)
>16 Features: 16, Balanced Accuracy: 0.924 (0.113)
>17 Features: 17, Balanced Accuracy: 0.924 (0.117)
>18 Features: 18, Balanced Accuracy: 0.926 (0.118)
>19 Features: 19, Balanced Accuracy: 0.935 (0.114)
>20 Features: 20, Balanced Accuracy: 0.935 (0.114

In [23]:
rfe = RFE(estimator=SVC(kernel='linear',
                    C=1.0734, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=2)
rfe.fit(data_1,target_1.ravel())
for i in range(data_1.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 17.000
Column: 1, Selected False, Rank: 6.000
Column: 2, Selected False, Rank: 16.000
Column: 3, Selected False, Rank: 11.000
Column: 4, Selected False, Rank: 21.000
Column: 5, Selected False, Rank: 14.000
Column: 6, Selected False, Rank: 8.000
Column: 7, Selected False, Rank: 22.000
Column: 8, Selected False, Rank: 13.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected False, Rank: 12.000
Column: 12, Selected False, Rank: 4.000
Column: 13, Selected False, Rank: 19.000
Column: 14, Selected False, Rank: 15.000
Column: 15, Selected False, Rank: 3.000
Column: 16, Selected False, Rank: 18.000
Column: 17, Selected False, Rank: 25.000
Column: 18, Selected False, Rank: 23.000
Column: 19, Selected False, Rank: 24.000
Column: 20, Selected False, Rank: 26.000
Column: 21, Selected False, Rank: 10.000
Column: 22, Selected False, Rank: 20.000
Column: 23, Selected False, Rank: 7.000
Column: 24, Selected False, Rank: 2

In [24]:
# Load feature names from the second dataset
labels_dat = pd.read_csv("240128-NTA-Normalized-Labels-Feature.csv")
feature_names = labels_dat.columns

# Print selected features with their names
selected_feature_indices = [i for i in range(len(rfe.support_)) if rfe.support_[i]]

for index in selected_feature_indices:
    print('Feature Name: %s, Index: %d, Rank: %.3f' % (feature_names[index], index, rfe.ranking_[index]))

Feature Name: 341.160715/9.2, Index: 9, Rank: 1.000
Feature Name: 251.164574/8.66, Index: 10, Rank: 1.000


In [25]:
selected_features_df = pd.DataFrame({
    'Feature Name': [feature_names[index] for index in selected_feature_indices]
})

# Export the DataFrame to an Excel file
selected_features_df.to_excel("RFE_selected_features_norm_LL_SVC.xlsx", index=False)

### BL

In [26]:
data_rf = pd.read_csv(r'240617-NTA-AVG-EUC-30-CLOSEST-FEATURE-logT.csv', header=0) #Targets: 92 samples X 581 features
#del data_rf[data_rf.columns[0]] #Dropping sample information
#print(data_rf)

In [27]:
#Prompt user for source type of interest (AFFF-GW, LF, BSL, WWTP, PP or PG)
preferred_type = input("Enter the source type of interest: ")

Enter the source type of interest: BL


In [28]:
#Manipulating data frame based on user input to make "Type" column read 1 for all samples of source of interest and 0 for all other samples
#Set up for binary classification (one-vs-all format)

# Define a function to apply to each row
def set_type(row):
    if row['Type'] == preferred_type:
        return 1
    else:
        return 0

# Create a new column "Type 2" with the updated values
data_rf['Type_2'] = data_rf.apply(set_type, axis=1)
del data_rf[data_rf.columns[0]] #Dropping original type column
#Reordering columns with Type_2 as first column
cols = list(data_rf.columns)
cols = [cols[-1]] + cols[:-1]
data_rf = data_rf[cols]

# Save the updated DataFrame to a new CSV file (if needed)
data_rf.to_csv('sample_data_with_labels_NEW10.csv', index=False)

In [29]:
#Changing pandas data frame to numpy for use in ML
data_rf_np = data_rf.to_numpy()
target_1 = data_rf_np[:,0].reshape(-1,1) #Convert target variables to 2D-array for sci-kit learn
data_1 = data_rf_np[:,1:]

#class_names=np.array([0.0,1.0])
#print(data_1.shape)
print(data_1)
#data_1 = pd.DataFrame(data_1)
#data_1.to_csv('log10_dat.csv', index=False)

[[2.54570928 2.54570928 4.11539313 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 4.54451372 4.87498812 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 3.22038648 4.8459347  ... 2.54570928 2.54570928 2.54570928]
 ...
 [2.54570928 2.54570928 2.54570928 ... 3.02475995 2.54570928 2.54570928]
 [2.54570928 2.54570928 2.54570928 ... 3.2965849  2.54570928 2.93484187]
 [2.54570928 2.54570928 3.58667926 ... 3.33173239 2.54570928 2.54570928]]


In [30]:
#ll with SVC
from sklearn.svm import SVC
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(1, 31):
        rfe = RFE(estimator=SVC(kernel='linear',
                    C=0.79648, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=i)
        model = SVC(kernel='linear',
                    C=0.79648, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

#Evaluate model
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, data_1, target_1, scoring='balanced_accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [31]:
# Get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, data_1, target_1)
    results.append(scores)
    names.append(name)
    print('>%s Features: %d, Balanced Accuracy: %.3f (%.3f)' % (name, int(name), np.mean(scores), np.std(scores)))

>1 Features: 1, Balanced Accuracy: 0.818 (0.214)
>2 Features: 2, Balanced Accuracy: 0.838 (0.197)
>3 Features: 3, Balanced Accuracy: 0.869 (0.190)
>4 Features: 4, Balanced Accuracy: 0.871 (0.192)
>5 Features: 5, Balanced Accuracy: 0.854 (0.201)
>6 Features: 6, Balanced Accuracy: 0.827 (0.202)
>7 Features: 7, Balanced Accuracy: 0.823 (0.203)
>8 Features: 8, Balanced Accuracy: 0.833 (0.184)
>9 Features: 9, Balanced Accuracy: 0.835 (0.185)
>10 Features: 10, Balanced Accuracy: 0.843 (0.175)
>11 Features: 11, Balanced Accuracy: 0.862 (0.167)
>12 Features: 12, Balanced Accuracy: 0.891 (0.153)
>13 Features: 13, Balanced Accuracy: 0.893 (0.154)
>14 Features: 14, Balanced Accuracy: 0.896 (0.155)
>15 Features: 15, Balanced Accuracy: 0.860 (0.182)
>16 Features: 16, Balanced Accuracy: 0.863 (0.183)
>17 Features: 17, Balanced Accuracy: 0.871 (0.183)
>18 Features: 18, Balanced Accuracy: 0.871 (0.183)
>19 Features: 19, Balanced Accuracy: 0.875 (0.173)
>20 Features: 20, Balanced Accuracy: 0.890 (0.172

In [36]:
rfe = RFE(estimator=SVC(kernel='linear',
                    C=0.79648, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=3)
rfe.fit(data_1,target_1.ravel())
for i in range(data_1.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 19.000
Column: 1, Selected False, Rank: 14.000
Column: 2, Selected False, Rank: 3.000
Column: 3, Selected False, Rank: 28.000
Column: 4, Selected False, Rank: 26.000
Column: 5, Selected False, Rank: 22.000
Column: 6, Selected False, Rank: 23.000
Column: 7, Selected False, Rank: 5.000
Column: 8, Selected False, Rank: 21.000
Column: 9, Selected False, Rank: 11.000
Column: 10, Selected False, Rank: 6.000
Column: 11, Selected False, Rank: 20.000
Column: 12, Selected False, Rank: 27.000
Column: 13, Selected False, Rank: 10.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected True, Rank: 1.000
Column: 17, Selected False, Rank: 16.000
Column: 18, Selected False, Rank: 17.000
Column: 19, Selected False, Rank: 13.000
Column: 20, Selected False, Rank: 25.000
Column: 21, Selected False, Rank: 12.000
Column: 22, Selected False, Rank: 4.000
Column: 23, Selected False, Rank: 24.000
Column: 24, Selected False, Rank: 7.

In [37]:
# Load feature names from the second dataset
labels_dat = pd.read_csv("240128-NTA-Normalized-Labels-Feature.csv")
feature_names = labels_dat.columns

# Print selected features with their names
selected_feature_indices = [i for i in range(len(rfe.support_)) if rfe.support_[i]]

for index in selected_feature_indices:
    print('Feature Name: %s, Index: %d, Rank: %.3f' % (feature_names[index], index, rfe.ranking_[index]))

Feature Name: 233.153427/11.74, Index: 14, Rank: 1.000
Feature Name: 301.215668/17.74, Index: 15, Rank: 1.000
Feature Name: 295.263376/19.32, Index: 16, Rank: 1.000


### PP

In [38]:
data_rf = pd.read_csv(r'240617-NTA-AVG-EUC-30-CLOSEST-FEATURE-logT.csv', header=0) #Targets: 92 samples X 581 features
#del data_rf[data_rf.columns[0]] #Dropping sample information
#print(data_rf)

In [39]:
#Prompt user for source type of interest (AFFF-GW, LF, BSL, WWTP, PP or PG)
preferred_type = input("Enter the source type of interest: ")

Enter the source type of interest: PP


In [40]:
#Manipulating data frame based on user input to make "Type" column read 1 for all samples of source of interest and 0 for all other samples
#Set up for binary classification (one-vs-all format)

# Define a function to apply to each row
def set_type(row):
    if row['Type'] == preferred_type:
        return 1
    else:
        return 0

# Create a new column "Type 2" with the updated values
data_rf['Type_2'] = data_rf.apply(set_type, axis=1)
del data_rf[data_rf.columns[0]] #Dropping original type column
#Reordering columns with Type_2 as first column
cols = list(data_rf.columns)
cols = [cols[-1]] + cols[:-1]
data_rf = data_rf[cols]

# Save the updated DataFrame to a new CSV file (if needed)
data_rf.to_csv('sample_data_with_labels_NEW10.csv', index=False)

In [41]:
#Changing pandas data frame to numpy for use in ML
data_rf_np = data_rf.to_numpy()
target_1 = data_rf_np[:,0].reshape(-1,1) #Convert target variables to 2D-array for sci-kit learn
data_1 = data_rf_np[:,1:]

#class_names=np.array([0.0,1.0])
#print(data_1.shape)
print(data_1)
#data_1 = pd.DataFrame(data_1)
#data_1.to_csv('log10_dat.csv', index=False)

[[2.54570928 2.54570928 4.11539313 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 4.54451372 4.87498812 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 3.22038648 4.8459347  ... 2.54570928 2.54570928 2.54570928]
 ...
 [2.54570928 2.54570928 2.54570928 ... 3.02475995 2.54570928 2.54570928]
 [2.54570928 2.54570928 2.54570928 ... 3.2965849  2.54570928 2.93484187]
 [2.54570928 2.54570928 3.58667926 ... 3.33173239 2.54570928 2.54570928]]


In [42]:
#ll with SVC
from sklearn.svm import SVC
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(1, 31):
        rfe = RFE(estimator=SVC(kernel='linear',
                    C=0.28184, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=i)
        model = SVC(kernel='linear',
                    C=0.28184, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

#Evaluate model
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, data_1, target_1, scoring='balanced_accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [43]:
# Get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, data_1, target_1)
    results.append(scores)
    names.append(name)
    print('>%s Features: %d, Balanced Accuracy: %.3f (%.3f)' % (name, int(name), np.mean(scores), np.std(scores)))

>1 Features: 1, Balanced Accuracy: 0.864 (0.193)
>2 Features: 2, Balanced Accuracy: 0.837 (0.199)
>3 Features: 3, Balanced Accuracy: 0.867 (0.191)
>4 Features: 4, Balanced Accuracy: 0.867 (0.191)
>5 Features: 5, Balanced Accuracy: 0.908 (0.151)
>6 Features: 6, Balanced Accuracy: 0.908 (0.164)
>7 Features: 7, Balanced Accuracy: 0.925 (0.146)
>8 Features: 8, Balanced Accuracy: 0.917 (0.149)
>9 Features: 9, Balanced Accuracy: 0.908 (0.164)
>10 Features: 10, Balanced Accuracy: 0.883 (0.180)
>11 Features: 11, Balanced Accuracy: 0.883 (0.180)
>12 Features: 12, Balanced Accuracy: 0.883 (0.180)
>13 Features: 13, Balanced Accuracy: 0.892 (0.167)
>14 Features: 14, Balanced Accuracy: 0.892 (0.167)
>15 Features: 15, Balanced Accuracy: 0.892 (0.167)
>16 Features: 16, Balanced Accuracy: 0.892 (0.167)
>17 Features: 17, Balanced Accuracy: 0.875 (0.180)
>18 Features: 18, Balanced Accuracy: 0.875 (0.180)
>19 Features: 19, Balanced Accuracy: 0.875 (0.180)
>20 Features: 20, Balanced Accuracy: 0.875 (0.180

In [49]:
rfe = RFE(estimator=SVC(kernel='linear',
                    C=0.28184, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=4)
rfe.fit(data_1,target_1.ravel())
for i in range(data_1.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 6.000
Column: 1, Selected False, Rank: 27.000
Column: 2, Selected False, Rank: 2.000
Column: 3, Selected False, Rank: 7.000
Column: 4, Selected False, Rank: 9.000
Column: 5, Selected False, Rank: 8.000
Column: 6, Selected False, Rank: 11.000
Column: 7, Selected False, Rank: 12.000
Column: 8, Selected False, Rank: 19.000
Column: 9, Selected False, Rank: 26.000
Column: 10, Selected False, Rank: 15.000
Column: 11, Selected False, Rank: 24.000
Column: 12, Selected False, Rank: 22.000
Column: 13, Selected False, Rank: 10.000
Column: 14, Selected False, Rank: 4.000
Column: 15, Selected False, Rank: 23.000
Column: 16, Selected False, Rank: 20.000
Column: 17, Selected False, Rank: 14.000
Column: 18, Selected False, Rank: 21.000
Column: 19, Selected False, Rank: 16.000
Column: 20, Selected False, Rank: 18.000
Column: 21, Selected False, Rank: 17.000
Column: 22, Selected False, Rank: 25.000
Column: 23, Selected False, Rank: 3.000
Column: 24, Selected False, Rank:

In [50]:
# Load feature names from the second dataset
labels_dat = pd.read_csv("240128-NTA-Normalized-Labels-Feature.csv")
feature_names = labels_dat.columns

# Print selected features with their names
selected_feature_indices = [i for i in range(len(rfe.support_)) if rfe.support_[i]]

for index in selected_feature_indices:
    print('Feature Name: %s, Index: %d, Rank: %.3f' % (feature_names[index], index, rfe.ranking_[index]))

Feature Name: 389.104431/8.13, Index: 25, Rank: 1.000
Feature Name: 451.160341/6.26, Index: 26, Rank: 1.000
Feature Name: 433.222417/8.54, Index: 28, Rank: 1.000
Feature Name: 435.201572/7.04, Index: 29, Rank: 1.000


In [48]:
selected_features_df = pd.DataFrame({
    'Feature Name': [feature_names[index] for index in selected_feature_indices]
})

# Export the DataFrame to an Excel file
selected_features_df.to_excel("RFE_selected_features_norm_PP_SVC.xlsx", index=False)

### PG

In [51]:
data_rf = pd.read_csv(r'240617-NTA-AVG-EUC-30-CLOSEST-FEATURE-logT.csv', header=0) #Targets: 92 samples X 581 features
#del data_rf[data_rf.columns[0]] #Dropping sample information
#print(data_rf)

In [52]:
#Prompt user for source type of interest (AFFF-GW, LF, BSL, WWTP, PP or PG)
preferred_type = input("Enter the source type of interest: ")

Enter the source type of interest: PG


In [53]:
#Manipulating data frame based on user input to make "Type" column read 1 for all samples of source of interest and 0 for all other samples
#Set up for binary classification (one-vs-all format)

# Define a function to apply to each row
def set_type(row):
    if row['Type'] == preferred_type:
        return 1
    else:
        return 0

# Create a new column "Type 2" with the updated values
data_rf['Type_2'] = data_rf.apply(set_type, axis=1)
del data_rf[data_rf.columns[0]] #Dropping original type column
#Reordering columns with Type_2 as first column
cols = list(data_rf.columns)
cols = [cols[-1]] + cols[:-1]
data_rf = data_rf[cols]

# Save the updated DataFrame to a new CSV file (if needed)
data_rf.to_csv('sample_data_with_labels_NEW10.csv', index=False)

In [54]:
#Changing pandas data frame to numpy for use in ML
data_rf_np = data_rf.to_numpy()
target_1 = data_rf_np[:,0].reshape(-1,1) #Convert target variables to 2D-array for sci-kit learn
data_1 = data_rf_np[:,1:]

#class_names=np.array([0.0,1.0])
#print(data_1.shape)
print(data_1)
#data_1 = pd.DataFrame(data_1)
#data_1.to_csv('log10_dat.csv', index=False)

[[2.54570928 2.54570928 4.11539313 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 4.54451372 4.87498812 ... 2.54570928 2.54570928 2.54570928]
 [2.54570928 3.22038648 4.8459347  ... 2.54570928 2.54570928 2.54570928]
 ...
 [2.54570928 2.54570928 2.54570928 ... 3.02475995 2.54570928 2.54570928]
 [2.54570928 2.54570928 2.54570928 ... 3.2965849  2.54570928 2.93484187]
 [2.54570928 2.54570928 3.58667926 ... 3.33173239 2.54570928 2.54570928]]


In [55]:
#ll with SVC
from sklearn.svm import SVC
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(1, 31):
        rfe = RFE(estimator=SVC(kernel='linear',
                    C=16.82827, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=i)
        model = SVC(kernel='linear',
                    C=16.82827, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

#Evaluate model
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=1)
    scores = cross_val_score(model, data_1, target_1, scoring='balanced_accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [56]:
# Get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, data_1, target_1)
    results.append(scores)
    names.append(name)
    print('>%s Features: %d, Balanced Accuracy: %.3f (%.3f)' % (name, int(name), np.mean(scores), np.std(scores)))

>1 Features: 1, Balanced Accuracy: 0.688 (0.180)
>2 Features: 2, Balanced Accuracy: 0.688 (0.180)
>3 Features: 3, Balanced Accuracy: 0.736 (0.195)
>4 Features: 4, Balanced Accuracy: 0.755 (0.205)
>5 Features: 5, Balanced Accuracy: 0.812 (0.179)
>6 Features: 6, Balanced Accuracy: 0.815 (0.147)
>7 Features: 7, Balanced Accuracy: 0.812 (0.150)
>8 Features: 8, Balanced Accuracy: 0.789 (0.147)
>9 Features: 9, Balanced Accuracy: 0.812 (0.148)
>10 Features: 10, Balanced Accuracy: 0.810 (0.142)
>11 Features: 11, Balanced Accuracy: 0.810 (0.179)
>12 Features: 12, Balanced Accuracy: 0.806 (0.182)
>13 Features: 13, Balanced Accuracy: 0.793 (0.180)
>14 Features: 14, Balanced Accuracy: 0.795 (0.180)
>15 Features: 15, Balanced Accuracy: 0.815 (0.188)
>16 Features: 16, Balanced Accuracy: 0.813 (0.186)
>17 Features: 17, Balanced Accuracy: 0.834 (0.191)
>18 Features: 18, Balanced Accuracy: 0.836 (0.193)
>19 Features: 19, Balanced Accuracy: 0.834 (0.186)
>20 Features: 20, Balanced Accuracy: 0.859 (0.160

In [61]:
rfe = RFE(estimator=SVC(kernel='linear',
                    C=16.82827, 
                    
                    tol=0.00001, 
                    shrinking=True, 
                    cache_size=200, 
                    verbose=False, 
                    max_iter=-1, 
                    probability=True), n_features_to_select=5)
rfe.fit(data_1,target_1.ravel())
for i in range(data_1.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 2.000
Column: 1, Selected False, Rank: 26.000
Column: 2, Selected False, Rank: 6.000
Column: 3, Selected False, Rank: 18.000
Column: 4, Selected False, Rank: 17.000
Column: 5, Selected False, Rank: 13.000
Column: 6, Selected False, Rank: 19.000
Column: 7, Selected False, Rank: 4.000
Column: 8, Selected False, Rank: 5.000
Column: 9, Selected False, Rank: 25.000
Column: 10, Selected False, Rank: 15.000
Column: 11, Selected False, Rank: 21.000
Column: 12, Selected False, Rank: 14.000
Column: 13, Selected False, Rank: 9.000
Column: 14, Selected False, Rank: 10.000
Column: 15, Selected False, Rank: 11.000
Column: 16, Selected False, Rank: 22.000
Column: 17, Selected True, Rank: 1.000
Column: 18, Selected True, Rank: 1.000
Column: 19, Selected True, Rank: 1.000
Column: 20, Selected False, Rank: 7.000
Column: 21, Selected False, Rank: 12.000
Column: 22, Selected False, Rank: 24.000
Column: 23, Selected False, Rank: 23.000
Column: 24, Selected False, Rank: 20.0

In [62]:
# Load feature names from the second dataset
labels_dat = pd.read_csv("240128-NTA-Normalized-Labels-Feature.csv")
feature_names = labels_dat.columns

# Print selected features with their names
selected_feature_indices = [i for i in range(len(rfe.support_)) if rfe.support_[i]]

for index in selected_feature_indices:
    print('Feature Name: %s, Index: %d, Rank: %.3f' % (feature_names[index], index, rfe.ranking_[index]))

Feature Name: 126.904941/20.82, Index: 17, Rank: 1.000
Feature Name: 429.223492/10.13, Index: 18, Rank: 1.000
Feature Name: 397.330809/15.57, Index: 19, Rank: 1.000
Feature Name: 389.104431/8.13, Index: 25, Rank: 1.000
Feature Name: 435.201572/7.04, Index: 29, Rank: 1.000


In [63]:
selected_features_df = pd.DataFrame({
    'Feature Name': [feature_names[index] for index in selected_feature_indices]
})

# Export the DataFrame to an Excel file
selected_features_df.to_excel("RFE_selected_features_norm_PG_SVC.xlsx", index=False)